In [1]:
#必要ライブラリ
# !pip install pandas
# !pip install beautifulsoup4
# !pip install selenium

     ---------------------------------------- 6.3/6.3 MB 10.1 MB/s eta 0:00:00
     ------------------------------------- 384.9/384.9 kB 12.1 MB/s eta 0:00:00
     ---------------------------------------- 58.3/58.3 kB ? eta 0:00:00


In [1]:
import pandas as pd
import time
import os
import re
import glob
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

In [20]:
#タイム/取得ページ数設定
waitTime=7
itemNumber=100 #100が入る
pageNumber=10 #何ページかが入る。多くてもあれなので10Pにしとく。後々は検索件数/100でわかる

▼まずは各項目を取得する関数を定義（わざわざ関数にしなくても良いけど後々楽になる）

In [76]:
#部屋名リスト取得(xpath利用)後々欠損値が出た時のためにこちらが有利
def get_roomName():
    roomNames_text=[]
    roomNames=driver.find_elements(By.CLASS_NAME, "js-cassetLinkHref")
    for i in range(len(roomNames)):
        roomNames_text.append(roomNames[i].text.replace("\u3000",""))
    return roomNames_text

In [87]:
#家賃/管理費リスト取得(xpath利用)
def get_roomPrice():
    roomPrices_text=[]
    roomPrices=driver.find_elements(By.CLASS_NAME, "detailbox-property--col1")
    for i in range(len(roomPrices)):
        roomPrices_text.append(roomPrices[i].text.replace("\u3000","").replace('管理費','').replace("-","0").replace("万","").replace("円","").split('\n')) #書き換え方がすごく乱暴な気もするがスプシに入れてから置換すると面倒なので先にやる 
    return roomPrices_text

In [79]:
#間取り/専有面積/方角/種別/築年数リスト取得
def get_roomArea():
    roomAreas_text=[]
    roomAreas=driver.find_elements(By.CLASS_NAME, "detailbox-property--col3")
    for i in range(len(roomAreas)):
        roomAreas_text.append(roomAreas[i].text.replace("\u3000","").replace("m2","").replace("築","").replace("年","").replace("新","0").replace("以上","").split('\n')) #新築は新と表示されるので0に書き換える。99年以上は99ということにする
        roomAreas_text[i][1]=round(float(roomAreas_text[i][1]))
    return roomAreas_text


In [73]:
round(float(roomAreas[0][1]))

45

In [56]:
# サイトへのアクセス
def get_website(url):
    driver.get(url) #後でiに変更
    time.sleep(waitTime)


▼実際に情報を取得していく。
１００件/ページ　ごとにリストを作成and更新していく。

In [80]:
#　SUUMO愛知県一宮市_賃貸_部屋ごとに表示_100件ずつ_最初のページ
url= "https://suumo.jp/jj/chintai/ichiran/FR301FC005/?ar=050&bs=040&ta=23&sc=23203&sngz=&po1=25&po2=99&pc=100"
#　2ページ目以降はurl末尾に&page=2をつければよい
page_list=[url]
for i in range(pageNumber):
    i+=2
    page_list.append(url+"&page="+str(i))
#　ブラウザ起動
driver = webdriver.Chrome(os.path.join(os.getcwd(), "/chromedriver.exe"))
time.sleep(waitTime)

#メインの格納データフレームを作成
#columns=["部屋名","家賃","専有面積","間取り"]

roomNames=[]
roomPrices=[]
roomAreas=[]
#　メイン実行
for i in range(pageNumber):
    get_website(page_list[i])
    roomNames.extend(get_roomName())
    roomPrices.extend(get_roomPrice())
    roomAreas.extend(get_roomArea())

C:\Users\kazum\AppData\Local\Temp\ipykernel_9052\3142931534.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(os.path.join(os.getcwd(), "/chromedriver.exe"))


▼作成したリストからデータフレーム（df）を作成していく。
今回データの前処理はあえて行わず何のデータかわかりやすい形で出力することを目指す。<br>
と思ったらスプシからデータを落としたときにReplaceがうまくできない事態が起きたのでスプシに入れる前に前処理をする。

In [81]:
#物件名のdf作成
columns_roomNames=["物件名"]
df_roomNames=pd.DataFrame(roomNames,columns=columns_roomNames)

In [82]:
#roomAreasの中身がブロックごと２リストずつ作られてしまうのでを結合する
#["間取り","専有面積"]+["方角","種別","築年数"]
roomAreas_unite=[]
i=0
while i<len(roomAreas):
    roomAreas_unite.append(roomAreas[i]+roomAreas[i+1])
    i+=2
#間取り/専有面積/方角/種別/築年数のｄｆ作成
columns_roomAreas=["間取り","専有面積","方角","種別","築年数"]
df_roomAreas=pd.DataFrame(roomAreas_unite,columns=columns_roomAreas)

In [83]:
#家賃/管理費のdf作成
columns_roomPrices=["家賃","管理費","備考"]
df_roomPrices=pd.DataFrame(roomPrices,columns=columns_roomPrices).drop(columns="備考") #余計な情報（備考）がついてくるので作ってすぐ消す

In [84]:
#それぞれのdfをまとめて一つのdfに
df=pd.concat([df_roomNames,df_roomPrices,df_roomAreas],axis=1)

▼dfがまとまったのを確認してcsvに出力。スプシに直接出力しても良いけど自分専用ではないので今回はcsvにする。

In [86]:
#csvに出力
df.to_csv("output.csv",index=False)
